### Grouping by cloudiness

Let's group Sentinel-2 images by how cloudy they are, and composite them into low-cloud and high-cloud composites. We'll look at a spot renowned for its clouds: Seattle.

`groupby` works on more than just dates; you can group by any metadata value on an Image.

You could even use it to group by `cloud_fraction`.
Though deriving any possible practical use of this is left as an exercise to the reader, it's still kind of fun to look at.

In [1]:
import descarteslabs.workflows as wf

/Users/gabejoseph/.local/share/virtualenvs/workflows-examples-ZimnatIE/lib/python3.6/site-packages/pkg_resources/__init__.py:1151: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  self, resource_name
/Users/gabejoseph/.local/share/virtualenvs/workflows-examples-ZimnatIE/lib/python3.6/site-packages/pkg_resources/__init__.py:1151: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  self, resource_name
/Users/gabejoseph/.local/share/virtualenvs/workflows-examples-ZimnatIE/lib/python3.6/site-packages/ipyleaflet/leaflet.py:316: DeprecationWarning: metadata {'dtype': None} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  data = Dataset().tag(dtype=None, sync=True, to_json=ds_x_to_json)


In [2]:
wf.map.center = [47.6857302113159, -122.19406127929689]
wf.map.zoom = 9

In [3]:
wf.map


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart Jupyter and re-run this notebook.


In [4]:
s2 = wf.ImageCollection.from_id('sentinel-2:L1C', start_datetime="2019-01-01")

In [5]:
grouped = s2.groupby(lambda img: img.properties['cloud_fraction'] // 0.25)
# cloud_fraction is 0-1, so this should give us up to 5 groups

In [6]:
low = grouped.groups[0].median(axis="images")
high = grouped.groups[3].mean(axis="images")
# notice we use `median` on clear images, `mean` on cloudy

In [7]:
low.pick_bands('red green blue').visualize('low', scales=[[0, 0.4], [0, 0.4], [0, 0.4]])
high.pick_bands('red green blue').visualize('high', scales=[[0, 1], [0, 1], [0, 1]])

-------

How many images fall into each group?

In [8]:
group_lengths = grouped.groups.items().map(
    lambda group_imgs: group_imgs[1].length()
)

In [9]:
group_lengths.compute(wf.map.geocontext())


Job ID: 6fd827202ad2df23a43f16bd861d573368433b15534730c3
[######] | Steps: 0/0 | Stage: DONE | Status: SUCCESS                         

[403, 192, 182, 530, 47]

What fraction of images have `cloud_fraction` <= 0.25?

In [11]:
clear_days_fraction = group_lengths[0] / (group_lengths.reduce(lambda accum, x: accum + x))

In [12]:
clear_days_fraction.compute(wf.map.geocontext())


Job ID: e6107ba04c4dd42f525eeac9e8b32b6808aee7c320ea8a60
[######] | Steps: 0/0 | Stage: DONE | Status: SUCCESS                         

0.29763663220088626

Indeed, it's cloudy a lot in Seattle.